# Experiment with finding dots in video using circular filters

Gary Bishop July 2018

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
from skimage.feature import peak_local_max

mpl.rcParams['figure.dpi']= 300

def show(im, **kwargs):
    '''Show images actual size unless it is tiny
    
    I'm assuming they are in LAB float32 if the rank is 3
    
    '''
    height, width = im.shape[:2]
    if height > 50 and width > 50:
        dpi = 100
        margin= 50
        figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
        left = margin/dpi/figsize[0] #axes ratio
        bottom = margin/dpi/figsize[1]

        fig = plt.figure(figsize=figsize, dpi=dpi)
        fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    else:
        plt.figure()
    
    args = dict(kwargs)
    if 'title' in args:
        del args['title']
    
    if len(im.shape) == 3:
        im = cv2.cvtColor(im, cv2.COLOR_LAB2RGB)
    elif len(im.shape) == 2:
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)
    if 'title' in kwargs:
        plt.title(kwargs['title'])
        
def isBlue(im):
    mblue = np.array([ 60.4 , -12.2, -35.7 ], dtype=np.float32)
    sblue = np.array([ 4.1, 3.2, 8.5], dtype=np.float32)
    d2 = np.sum((im - mblue)**2 / sblue**2, axis=2)
    return np.exp(-d2 / 10)

def circularity(contour):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    contour = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    result = 4 * np.pi * (area / perimeter ** 2)
    return result

def isCircular(contour, hull=False):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    if hull:
        contour = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    circularity = 4 * np.pi * (area / perimeter ** 2)
    return 0.7 <= circularity <= 1.2
    # return 0.5 <= circularity <= 1.4

In [ ]:
vid = '/home/gb/Dropbox/Karen and Gary Shared Files/Videos & Transcripts/MSB/MSB_Video 1 (09-30-17).mp4'
vc = cv2.VideoCapture(vid)
vc.get(cv2.CAP_PROP_FRAME_COUNT), vc.get(cv2.CAP_PROP_FPS), vc.get(cv2.CAP_PROP_FRAME_WIDTH), vc.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [ ]:
def grabFrame(fn):
    vc.set(cv2.CAP_PROP_POS_FRAMES, fn)
    rval, im = vc.read()
    im = cv2.cvtColor(im.astype(np.float32)/255.0, cv2.COLOR_BGR2LAB)
    return im
show(grabFrame(800))
plt.grid(1)

In [ ]:
def makeCircleFilter(radius, size):
    scaleup = 5
    ssize = scaleup * size
    c = (ssize - 1) / 2
    x, y = np.meshgrid(np.arange(ssize)-c, np.arange(ssize)-c)
    r = np.sqrt(x**2 + y**2)
    im = (r <= radius*scaleup).astype(np.float32)
    result = np.zeros((size, size), dtype=np.float32)
    for r in range(size):
        for c in range(size):
            result[r,c] = np.mean(im[r*scaleup:r*scaleup+scaleup, c*scaleup:c*scaleup+scaleup])
    return result / np.sum(result)
c = makeCircleFilter(15, 31)
show(c)
plt.figure()
plt.plot(c[10,:])

In [ ]:
im = grabFrame(1700)
#im = im[250:400,200:350,:]
bim = isBlue(im)
dbim = cv2.dilate(bim, kernel=np.ones((3,3)), iterations=1)
radii = np.arange(4,15,0.5)
fwh = 5
xyc = (fwh - 1) // 2
fs = 3
sc = (fs - 1) // 2
footprint = np.zeros((fwh,fwh,fs))
xysteps = np.arange(fwh) - xyc
ys, xs = np.meshgrid(xysteps, xysteps)
footprint[:,:,sc] = ys**2 + xs**2 < fwh**2/4
footprint[xyc,xyc,:] = 1
footprint[:,:,:] = 1
print(footprint)
stack = np.dstack([ cv2.filter2D(bim, -1, makeCircleFilter(radius, 31)) for radius in radii ])
dstack = np.dstack([ cv2.filter2D(dbim, -1, makeCircleFilter(radius, 31)) for radius in radii ])
coordinates = peak_local_max(stack, footprint=footprint, threshold_abs=0.5)
# filter by dstack threshold
circles =  [ (y, x, ndx, radii[ndx], dstack[y, x, ndx]) for y, x, ndx in coordinates if dstack[y, x, ndx] > 0.7 ]
for y, x, ndx, radius, value in circles:
    print(y, x, ndx, radius, value, value / np.min(stack[y, x, :]))
oim = im.copy()
for y, x, ndx, radius, value in circles:
    cv2.circle(oim, (x,y), int(radii[ndx]), (0,0,0), lineType=cv2.LINE_AA)
show(oim)
plt.grid(1)

In [ ]:
plt.plot(stack[59,74,:])

In [ ]:
plt.plot(stack[59,74,:])

In [ ]:
print(footprint[:,:,0])

In [ ]:
plt.imshow(bim, cmap='gray')
plt.grid('on')

In [ ]:
im = grabFrame(1700)
minRadius = 5.5
maxRadius = 15
kernelWidth = int(2*maxRadius+1)
radii = np.arange(minRadius,maxRadius,0.5)
circleFilters = [ makeCircleFilter(radius, kernelWidth) for radius in radii ]
footprint = np.ones((3,3,3))
def getCircles(im):
    bim = isBlue(im)
    #bim = cv2.dilate(bim, kernel=np.ones((3,3)), iterations=1)
    #bim = cv2.erode(bim, kernel=np.ones((3,3)), iterations=1)
    stack = np.dstack([ cv2.filter2D(bim, -1, kernel) for kernel in circleFilters ])
    coordinates = peak_local_max(stack, footprint=footprint, threshold_abs=0.3)
    return [ (y, x, ndx, radii[ndx], stack[y, x, ndx]) for y, x, ndx in coordinates ]
    
circles = getCircles(im)
for y, x, ndx, radius, value in circles:
    print(y, x, ndx, radius, value, value / np.min(stack[y, x, :]))
oim = im.copy()
for y, x, ndx, radius, value in circles:
    if value / np.min(stack[y, x, :]) > 2:
        cv2.circle(oim, (x,y), int(radii[ndx]), (0,0,0), lineType=cv2.LINE_AA)
show(oim)
plt.grid(1)
print(np.max(bim), np.max(stack))

In [ ]:
plt.imshow(stack[200:300,300:400,3], cmap='gray')
plt.grid(1)

In [ ]:
plt.plot(radii, stack[79,315,:])
plt.plot(radii, stack[355,241,:])

## Find all circles and save them away

In [ ]:
results = []
nframes = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
for fno in range(100, 1100, 1):
    if fno % 100 == 0:
        print(fno, len(results))
    frame = grabFrame(fno)
    for y, x, ndx, radius, value in getCircles(frame):
        r = int(radius+0.5)
        y1 = max(0, y-r)
        y2 = min(frame.shape[0], y+r+1)
        x1 = max(0, x-r)
        x2 = min(frame.shape[1], x+r+1)
        pixels = frame[y1:y2, x1:x2].copy()
        results.append((fno, y, x, radius, value, pixels))


In [ ]:
import pickle
pickle.dump(results, open('MSB_Video_1.circles', 'wb'))